<a href="https://colab.research.google.com/github/srujana-7/pySpark/blob/main/Word_Count_using_data_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  pySpark Word Count program:
  Steps followed-
1.Configure Google Colab to run pySpark
2.Upload data file from drive to colab
3.Convert all uppercase words to lower case and remove all punctuation.
4.Get the word count.






In [ ]:

#Configuring google colab with pySpark 
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [23]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [7]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
#Function to clean an input text file with punctuations and convert to lowercase 
def removePunctuation(column):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        column (Column): A Column containing a sentence.

    Returns:
        Column: A Column named 'sentence' with clean-up operations applied.
    """
    return trim(lower(regexp_replace(column, '([^\s\w_]|_)+', ''))).alias('sentence')




In [ ]:
# code to test function
sentenceDF = sqlContext.createDataFrame([('Hi,  This',),
                                         (' Is just to test above function!',),
                                         (' *  ---Check if these punctuations are removed---  * ',)], ['sentence'])
  # dispaly first original sentence
sentenceDF.show(truncate=False)
  # then sentence with punctuation removed
(sentenceDF
 .select(removePunctuation(col('sentence')))
 .show(truncate=False))

In [9]:
#IMPORT FILES FROM DRIVE INTO GOOGLE-COLAB:

#STEP-1: Import Libraries

# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#STEP-2: Autheticate E-Mail ID

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
#2.1 Get the file
downloaded = drive.CreateFile({'id':'17neskYlVTp7q4_T4_TpM9GM6VoRR2pp7'})
 # replace the id with id of file you want to access , which is found on google drive.
#On the file on your drive, right click on the file and click get shareable link , copy paste
#Remove the part of the link other than ID
downloaded.GetContentFile('Word_Count_Data.txt') 


In [ ]:
fileName = "Word_Count_Data.txt"

DF = sqlContext.read.text(fileName).select(removePunctuation(col('value')))
DF.show(15, truncate=False)
DF.count()

In [ ]:
from pyspark.sql.functions import split, explode

DF2 = (DF .select(split(DF.sentence, '\s+').alias('split')))
DF3 = (DF2.select(explode(DF2.split).alias('word')))

DF3.show()

In [ ]:
wordListDF= DF3.groupBy('word').count()

wordListDF.show(100)
wordListDF.count()